In [ ]:
!pip install ollama colab-xterm -q
%load_ext colabxterm

#curl https://ollama.ai/install.sh | sh
# ollama serve &
# ollama pull qwen3-vl:4b

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 12.2 MB/s eta 0:00:00


# imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [41]:
import os
import numpy as np
import pandas as pd
import json
import cv2
import re

In [ ]:
prompt = "You are a careful visual annotator. Look only at the image. Decide what kind of scene is shown using the categories below. Be conservative: if unclear, use \"scene_type\": \"other_or_unknown\". Output JSON ONLY. \n\nScene type options (decide based on visible cues): \n- combat_or_military_action: Weapons, explosions, airstrikes, armed soldiers in action or at checkpoints. \n- destruction_or_humanitarian_crisis: Rubble, collapsed buildings, smoke, damaged streets, tents or shelters, refugees, queues for aid, doctors or rescuers helping civilians. \n- political_or_diplomatic_event: Politicians or officials at podiums or in formal meetings, parliaments, press rooms, negotiation tables, government ceremonies. \n- news_media_or_interview: TV anchors in a studio, reporters speaking to camera, people being interviewed with a microphone, talk-show or split-screen news formats. \n- public_protest_or_demonstration: Crowds holding signs or banners, marches, rallies, vigils in streets or squares, police lines facing demonstrators (including protests outside the conflict region). \n- symbolic_or_religious_ritual: Religious buildings or interiors, prayer, clergy, funerals, coffins, memorials, monuments, candlelight vigils, large flags used in a ceremonial or ritual way. \n- other_or_unknown: Any scene that does not clearly match the above categories or is too ambiguous. Schema: {\"scene_type\": [combat_or_military_action|destruction_or_humanitarian_crisis|political_or_diplomatic_event|news_media_or_interview|public_protest_or_demonstration|symbolic_or_religious_ritual|other_or_unknown], text_overlay: present | absent | unknown, evidence: [<=2 short phrases describing visible cues]} If JSON output fails, reprompt with: \"Your previous output was invalid. Follow the schema exactly. Output JSON only.\""

# functions

In [ ]:
from ollama import chat
import base64
from PIL import Image
import imageio.v3 as iio
model = 'qwen3-vl:4b'

def run_ollama_chat(img, prompt):
  img_data = iio.imwrite("<bytes>", img, extension='.jpg')


  stream = chat(
      model=model,
      messages=[{'role': 'user', 'content': prompt,
        'images': [base64.b64encode(img_data).decode('utf-8')]}],
      stream=True,
  )
  response = ""
  for chunk in stream:
    response += chunk['message']['content']
  return response

# queries


In [18]:
images = []
total = 0

for image in os.listdir("drive/MyDrive/yt_project/qwen/frames/"):
  images.append(image)
  total += 1

In [ ]:
responses = []
x = 1

for image in images:
  if(len(responses)+1 <= x):
    img = iio.imread("drive/MyDrive/yt_project/qwen/frames/" + image)
    responses.append(run_ollama_chat(img, prompt))

  print(str(x) + "/" + str(total))
  x += 1

# compile csv

In [25]:
id = []
video_name = []

for outlet in os.listdir("drive/MyDrive/yt_project/qwen/videos/"):
  x = 0
  for video in os.listdir("drive/MyDrive/yt_project/qwen/videos/" + outlet):
    video_name.append(video)
    id.append(outlet + str(x))

    x += 1

video_id = list(zip(video_name, id))

In [47]:
video_infos = []
video_name, id = zip(*video_id)

for x in range(len(responses)):
  info = []
  info.append(video_name[id.index(images[x].split("_")[0])])
  info.append(images[x].split("_")[0])
  info.append(images[x].split("_")[0] + "_" + images[x].split("_")[1])

  try:
    a = json.loads(responses[x])
    info.append(a['scene_type'])
    info.append(a['text_overlay'])
    info.append(a['evidence'])
  except:
    info.append([0, 0, 0])

  video_infos.append(info)

In [49]:
(pd.DataFrame(video_infos)).to_csv('bbc_dw_scenes.csv')